In [2]:
import pyspark
import pyspark.sql
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
test = sqlContext.read.csv("data\\test_table.csv",header = True) # \t is tap; need to use \\

In [4]:
user = sqlContext.read.csv("data\\user_table.csv",header = True)

# SparkSQL 

## test file 

In [5]:
test.show()

+-------+----------+------+------+----------------+-----------+-----------+----------+----+
|user_id|      date|source|device|browser_language|ads_channel|    browser|conversion|test|
+-------+----------+------+------+----------------+-----------+-----------+----------+----+
| 315281|2015-12-03|Direct|   Web|              ES|         NA|         IE|         1|   0|
| 497851|2015-12-04|   Ads|   Web|              ES|     Google|         IE|         0|   1|
| 848402|2015-12-04|   Ads|   Web|              ES|   Facebook|     Chrome|         0|   0|
| 290051|2015-12-03|   Ads|Mobile|           Other|   Facebook|Android_App|         0|   1|
| 548435|2015-11-30|   Ads|   Web|              ES|     Google|    FireFox|         0|   1|
| 540675|2015-12-03|Direct|Mobile|              ES|         NA|Android_App|         0|   1|
| 863394|2015-12-04|   SEO|Mobile|           Other|         NA|Android_App|         0|   0|
| 527287|2015-12-03|Direct|   Web|              EN|         NA|     Chrome|     

In [6]:
# Number of Rows 
test.count()

453321

In [7]:
test.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- source: string (nullable = true)
 |-- device: string (nullable = true)
 |-- browser_language: string (nullable = true)
 |-- ads_channel: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- conversion: string (nullable = true)
 |-- test: string (nullable = true)



In [8]:
test.columns

['user_id',
 'date',
 'source',
 'device',
 'browser_language',
 'ads_channel',
 'browser',
 'conversion',
 'test']

In [9]:
def group_by(x):
    test.groupBy(x).count().show()
for i in test.columns:
    if i != 'user_id':
        print(i)
        group_by(i)

date
+----------+------+
|      date| count|
+----------+------+
|2015-12-03| 99493|
|2015-11-30| 71025|
|2015-12-02| 70649|
|2015-12-04|141163|
|2015-12-01| 70991|
+----------+------+

source
+------+------+
|source| count|
+------+------+
|   SEO|180610|
|Direct| 90834|
|   Ads|181877|
+------+------+

device
+------+------+
|device| count|
+------+------+
|Mobile|201756|
|   Web|251565|
+------+------+

browser_language
+----------------+------+
|browser_language| count|
+----------------+------+
|              EN| 63137|
|              ES|377547|
|           Other| 12637|
+----------------+------+

ads_channel
+-----------+------+
|ads_channel| count|
+-----------+------+
|         NA|271444|
|      Yahoo| 27435|
|       Bing| 13689|
|      Other|  4148|
|     Google| 68180|
|   Facebook| 68425|
+-----------+------+

browser
+-----------+------+
|    browser| count|
+-----------+------+
| Iphone_App| 46621|
|    FireFox| 40766|
|     Safari| 41065|
|Android_App|155135|
|         IE

Data Summary: 
1. [Date] The data records 5-day web transaction 
2. [Source] Most of the users came to the website through SEO or Ads
3. [device] Over half of the users visit the website through laptop or PC
4. [Brower_language] Most of the users browse the website using Spanish, in this case we only consider spanish-speaking users
5. [ads_channel] Google and Facebook ads is more effective than others 
6. [browser] Most of the users visite the website through Chrome and Android_APP
7. [conversion] 430846 users did not make transactions, only small proportion of users (22475) make transactions
8. [test] over half of the users did not take the test.We expect none of the Spain-based users and non-spanish users did not have the test

In [10]:
# Hypothesis: Non-spanish-speaking users are not necessary for the test 
# Fact: almost half of non-spanis-speaking users are in the test 
# Solutions: We should not take these users into account 
test.filter(test.browser_language != 'ES').groupBy('test').count().show()

+----+-----+
|test|count|
+----+-----+
|   0|39565|
|   1|36209|
+----+-----+



In [11]:
test.groupby(['test','conversion']).count().show()

+----+----------+------+
|test|conversion| count|
+----+----------+------+
|   0|         1| 13096|
|   1|         1|  9379|
|   0|         0|224242|
|   1|         0|206604|
+----+----------+------+



## user

In [12]:
user.columns

['user_id', 'sex', 'age', 'country']

In [13]:
user.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)



In [14]:
user.show(20)

+-------+---+---+---------+
|user_id|sex|age|  country|
+-------+---+---+---------+
| 765821|  M| 20|   Mexico|
| 343561|  F| 27|Nicaragua|
| 118744|  M| 23| Colombia|
| 987753|  F| 27|Venezuela|
| 554597|  F| 20|    Spain|
|  62371|  M| 29|  Ecuador|
| 987967|  M| 31|    Spain|
| 499510|  F| 29| Paraguay|
| 364726|  F| 29| Colombia|
| 572551|  M| 21|  Bolivia|
| 224416|  F| 49|    Spain|
| 601332|  M| 29|Argentina|
| 518243|  M| 24|    Spain|
| 585161|  M| 20|     Peru|
| 638738|  F| 22|Venezuela|
| 751480|  F| 18|   Mexico|
| 281279|  M| 37|  Ecuador|
| 692649|  M| 18|   Mexico|
| 966822|  F| 25|     Peru|
| 270794|  M| 26|    Spain|
+-------+---+---+---------+
only showing top 20 rows



In [15]:
user.count()

452867

In [16]:
for i in user.columns:
    if i !='user_id' and i != 'age':
        user.groupBy(i).count().orderBy('count',ascending = False).show()

+---+------+
|sex| count|
+---+------+
|  M|264485|
|  F|188382|
+---+------+

+-----------+------+
|    country| count|
+-----------+------+
|     Mexico|128484|
|   Colombia| 54060|
|      Spain| 51782|
|  Argentina| 46733|
|       Peru| 33666|
|  Venezuela| 32054|
|      Chile| 19737|
|    Ecuador| 15895|
|  Guatemala| 15125|
|    Bolivia| 11124|
|   Honduras|  8568|
|El Salvador|  8175|
|   Paraguay|  7347|
|  Nicaragua|  6723|
| Costa Rica|  5309|
|    Uruguay|  4134|
|     Panama|  3951|
+-----------+------+



Summary: 
1. [sex] Most of the users are men
2. [country] The biggest proportion of the user is from Mexico, Spain is the third country in terms of user population 

In [17]:
from pyspark.sql import functions as F
user.select(F.min("age")).show()
user.select(F.max("age")).show()

+--------+
|min(age)|
+--------+
|      18|
+--------+

+--------+
|max(age)|
+--------+
|      70|
+--------+



## Join two table together

In [46]:
df = test.join(user,on = 'user_id')

In [47]:
df.show()

+-------+----------+------+------+----------------+-----------+-----------+----------+----+---+---+---------+
|user_id|      date|source|device|browser_language|ads_channel|    browser|conversion|test|sex|age|  country|
+-------+----------+------+------+----------------+-----------+-----------+----------+----+---+---+---------+
| 100227|2015-12-04|   Ads|   Web|              ES|   Facebook|    FireFox|         0|   0|  M| 23|     Peru|
| 100263|2015-11-30|   SEO|Mobile|              ES|         NA|Android_App|         0|   0|  M| 28|    Spain|
| 100735|2015-12-01|   Ads|   Web|              ES|     Google|     Chrome|         0|   1|  F| 29|   Mexico|
| 101122|2015-12-02|   Ads|Mobile|              EN|     Google|Android_App|         0|   1|  F| 44|  Uruguay|
| 102113|2015-12-04|   Ads|Mobile|              ES|   Facebook|Android_App|         0|   1|  F| 33| Colombia|
| 102521|2015-12-04|   Ads|Mobile|              EN|   Facebook|Android_App|         0|   0|  M| 44|   Mexico|
| 102684|2

In [48]:
df.count()

452867

In [49]:
# we need to filter out the non-spanish-speaking users 
new_df = df.filter(df.browser_language == 'ES')
new_df.count()

377160

In [79]:
# User Population by country
pop_df = new_df.groupBy('country').count().orderBy('count',ascending = False)
pop_df.show()

+-----------+------+
|    country| count|
+-----------+------+
|     Mexico|107086|
|   Colombia| 44850|
|      Spain| 43146|
|  Argentina| 38997|
|       Peru| 28000|
|  Venezuela| 26733|
|      Chile| 16433|
|    Ecuador| 13262|
|  Guatemala| 12610|
|    Bolivia|  9254|
|   Honduras|  7167|
|El Salvador|  6775|
|   Paraguay|  6089|
|  Nicaragua|  5636|
| Costa Rica|  4378|
|    Uruguay|  3454|
|     Panama|  3290|
+-----------+------+



In [78]:
# Which country has a high conversion number? 
new_df.filter(df.conversion == '1').groupBy('country').count().orderBy('count',ascending = False).show()

+-----------+-----+
|    country|count|
+-----------+-----+
|     Mexico| 5403|
|      Spain| 3443|
|   Colombia| 2330|
|       Peru| 1398|
|  Venezuela| 1344|
|      Chile|  805|
|    Ecuador|  658|
|  Guatemala|  638|
|  Argentina|  559|
|    Bolivia|  458|
|   Honduras|  357|
|El Salvador|  351|
|  Nicaragua|  306|
|   Paraguay|  287|
| Costa Rica|  229|
|     Panama|  164|
|    Uruguay|   37|
+-----------+-----+



In [62]:
# without test, we want to caculate conversion rate by country
conversion = new_df.filter((df.test == '0') & (df.conversion == '1')).groupBy('country').count()

In [63]:
conversion = conversion.select(conversion['country'],conversion['count'].alias("Number of conversion"))


In [64]:
conversion_rate_df = new_df.filter(df.test == '0').groupBy('country').count().join(conversion, on = 'country')


In [70]:

without_test_df =\
conversion_rate_df\
.withColumn('Conversion rate without test', conversion_rate_df['Number of conversion']/conversion_rate_df['count'])\
.orderBy('Conversion rate without test', ascending = False)
without_test_df.show()

+-----------+-----+--------------------+----------------------------+
|    country|count|Number of conversion|Conversion rate without test|
+-----------+-----+--------------------+----------------------------+
|      Spain|43146|                3443|         0.07979882260232699|
|El Salvador| 3420|                 187|        0.054678362573099416|
|  Nicaragua| 2869|                 155|         0.05402579295921924|
|   Colombia|22502|                1190|        0.052884188072171365|
| Costa Rica| 2196|                 113|         0.05145719489981785|
|  Venezuela|13488|                 684|        0.050711743772241996|
|   Honduras| 3636|                 184|        0.050605060506050605|
|    Ecuador| 6746|                 340|         0.05040023717758672|
|     Mexico|53550|                2676|         0.04997198879551821|
|  Guatemala| 6373|                 318|         0.04989800721795073|
|    Bolivia| 4600|                 228|        0.049565217391304345|
|       Peru|13990| 

In [87]:
# with test, we want to caculate conversion rate by country
conversion_test = new_df.filter((df.test == '1') & (df.conversion == '1')).groupBy('country').count()
conversion_test = conversion_test.select(conversion_test['country'],conversion_test['count'].alias("Number of conversion"))
conversion_rate_test_df = new_df.filter(df.test == '1').groupBy('country').count().join(conversion_test, on = 'country')
with_test_df = \
conversion_rate_test_df\
.withColumn('Conversion rate with test', conversion_rate_test_df['Number of conversion']/conversion_rate_test_df['count'])\
.orderBy('Conversion rate with test', ascending = False)
with_test_df.show()

+-----------+-----+--------------------+-------------------------+
|    country|count|Number of conversion|Conversion rate with test|
+-----------+-----+--------------------+-------------------------+
|  Nicaragua| 2767|                 151|     0.054571738344777736|
| Costa Rica| 2182|                 116|      0.05316223648029331|
|     Panama| 1662|                  86|      0.05174488567990373|
|  Guatemala| 6237|                 320|      0.05130671797338464|
|   Colombia|22348|                1140|      0.05101127617683909|
|     Mexico|53536|                2727|      0.05093768679019725|
|       Peru|14010|                 707|      0.05046395431834404|
|  Venezuela|13245|                 660|      0.04983012457531144|
|      Chile| 8171|                 406|      0.04968792069514135|
|    Bolivia| 4654|                 230|      0.04941985388912763|
|   Honduras| 3531|                 173|     0.048994619088077035|
|El Salvador| 3355|                 164|       0.0488822652757

In [88]:
# comparing with two case, we found some countries with test has a better conversion rate, such as Mexico
comb_df = without_test_df.select('country','Conversion rate without test')\
.join(with_test_df.select('country','Conversion rate with test'), on = 'country',how = 'left')\
.join(pop_df, on = 'country')
comb_df.select('country',comb_df['count'].alias('population'),'Conversion rate without test','Conversion rate with test',\
               F.when(comb_df['Conversion rate with test']>comb_df['Conversion rate without test'],1)\
               .otherwise(0).alias('better resulte with test?'))\
         .orderBy('population',ascending = False).show()

+-----------+----------+----------------------------+-------------------------+-------------------------+
|    country|population|Conversion rate without test|Conversion rate with test|better resulte with test?|
+-----------+----------+----------------------------+-------------------------+-------------------------+
|     Mexico|    107086|         0.04997198879551821|      0.05093768679019725|                        1|
|   Colombia|     44850|        0.052884188072171365|      0.05101127617683909|                        0|
|      Spain|     43146|         0.07979882260232699|                     null|                        0|
|  Argentina|     38997|        0.015398434492493264|     0.014068709139853865|                        0|
|       Peru|     28000|         0.04939242315939957|      0.05046395431834404|                        1|
|  Venezuela|     26733|        0.050711743772241996|      0.04983012457531144|                        0|
|      Chile|     16433|        0.048293391430

Conclusion based on the conversion rate: 
    Ignoring Spain, we found 8 countries has a better result with the language version written by the local, and 8 countries have a worse result. Overall, I don't think that the test is negative is correct. We cannot judge whether this test is positive or negative based on the conversion rate, as the fluctuation is too small to consider. Thus, I don't think the problem is due to translation.  
    Additionally, Mexico, the country with most users, has a better result.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mp
pdf = new_df.select("*").toPandas()